# Определение стоимости автомобилей

В распоряжении имеются исторические данные: технические характеристики, комплектации и цены автомобилей. Необходимо построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

**Цель исследования** — построить модель с минимальным значением итоговой метрики RMSE, которая спрогнозирует рыночную стоимость автомобиля.

**Ход исследования**

Данные о технических характеристиках, комплектации и ценах других автомобилей будут получены из файла `autos.csv`. О качестве данных ничего не известно. Поэтому перед началом исследования понадобится обзор данных. 
 
План работы:
 1. Предобработка и подготовка данных.
 2. Обучение моделей.
 3. Анализ моделей.
 4. Тестирование.
 5. Итоги исследования.

## Подготовка данных

In [1]:
!pip install lightgbm

In [2]:
!pip install scikit-learn==1.1.3

     |████████████████████████████████| 30.8 MB 2.1 MB/s eta 0:00:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.1
    Uninstalling scikit-learn-0.24.1:
      Successfully uninstalled scikit-learn-0.24.1


In [3]:
# Импортируем необходимые библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb

from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error,make_scorer
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression

In [4]:
# сохраним данные в переменную
df = pd.read_csv('autos.csv')

In [5]:
# Посмотрим на таблицу с данными
df.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [6]:
# Выведем общую информацию о данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [7]:
# посмотрим на долю пропусков
pd.DataFrame(round(df.isna().mean()*100,1)).style.background_gradient('coolwarm')

,0
DateCrawled,0.000000
Price,0.000000
VehicleType,10.600000
RegistrationYear,0.000000
Gearbox,5.600000
Power,0.000000
Model,5.600000
Kilometer,0.000000
RegistrationMonth,0.000000
FuelType,9.300000


In [8]:
df.isna().sum()

DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Kilometer                0
RegistrationMonth        0
FuelType             32895
Brand                    0
Repaired             71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

In [9]:
# Количество полных дубликатов
df.duplicated().sum()

4

In [10]:
# Удалим полные дубликаты
df = df.drop_duplicates().reset_index(drop=True)

In [11]:
# Поменяем формат признаков с датами на datetime
df['DateCrawled'] = pd.to_datetime(df['DateCrawled'], format='%Y-%m-%dT%H:%M:%S')
df['DateCreated'] = pd.to_datetime(df['DateCreated'], format='%Y-%m-%dT%H:%M:%S')
df['LastSeen'] = pd.to_datetime(df['LastSeen'], format='%Y-%m-%dT%H:%M:%S')

In [12]:
# Выведем статистические данные о признаках с датами
df[['DateCrawled','DateCreated','LastSeen']].describe(datetime_is_numeric=True)

,DateCrawled,DateCreated,LastSeen
count,354365,354365,354365
mean,2016-03-21 12:58:09.520460544,2016-03-20 19:12:06.583042560,2016-03-29 23:51:06.766184960
min,2016-03-05 14:06:22,2014-03-10 00:00:00,2016-03-05 14:15:08
25%,2016-03-13 11:52:33,2016-03-13 00:00:00,2016-03-23 02:50:03
50%,2016-03-21 17:50:55,2016-03-21 00:00:00,2016-04-03 15:15:52
75%,2016-03-29 14:37:20,2016-03-29 00:00:00,2016-04-06 10:15:19
max,2016-04-07 14:36:58,2016-04-07 00:00:00,2016-04-07 14:58:51


Мы видим, что последнее извлечение данных из базы было 2016-04-07 14:36:58.

**Исследуем количественные признаки**

In [13]:
# Посмотрим на корреляцию между данными
df.corr()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
Price,1.000000,0.026916,0.158872,-0.333207,0.110580,NaN,0.076058
RegistrationYear,0.026916,1.000000,-0.000828,-0.053448,-0.011619,NaN,-0.003459
Power,0.158872,-0.000828,1.000000,0.024006,0.043379,NaN,0.021662
Kilometer,-0.333207,-0.053448,0.024006,1.000000,0.009575,NaN,-0.007685
RegistrationMonth,0.110580,-0.011619,0.043379,0.009575,1.000000,NaN,0.013996
NumberOfPictures,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PostalCode,0.076058,-0.003459,0.021662,-0.007685,0.013996,NaN,1.000000


In [14]:
# Исследуем цены автомобилей
df[['Price']].describe()

,Price
count,354365.000000
mean,4416.679830
std,4514.176349
min,0.000000
25%,1050.000000
50%,2700.000000
75%,6400.000000
max,20000.000000


In [15]:
df = df[df['Price'] >= 1] # оставим только автомобили с ценами >= 1

In [16]:
# Исследуем год регистрации объектов
df[['RegistrationYear']].describe()

,RegistrationYear
count,343593.00000
mean,2004.08983
std,78.41368
min,1000.00000
25%,1999.00000
50%,2003.00000
75%,2008.00000
max,9999.00000


Так как последняя дата скачивания анкеты из базы была 2016-04-07, то удалим автомобили зарегистрированные позже 2016 года (это не имеет смысла). Также удалим объекты зарегистрированные до 1950 года, они уже могут являться раритетнными и их рыночная стоимость может значительно отличаться.

In [17]:
df = df[(df['RegistrationYear'] <= 2016) & (df['RegistrationYear'] >= 1950)] 

In [18]:
# Исследуем мощность
df[['Power']].describe()

,Power
count,329635.000000
mean,111.931806
std,184.862436
min,0.000000
25%,71.000000
50%,105.000000
75%,143.000000
max,20000.000000


Заменим выбросы мощности больше 750 л.с. и аномалии менее 10 л.с. медианным значением.

In [19]:
df['Power'] = df['Power'].where((df['Power'] <= 750) & (df['Power'] >= 10), 105)

In [20]:
# Исследуем пробег
df[['Kilometer']].describe()

,Kilometer
count,329635.000000
mean,128232.620929
std,37487.625858
min,5000.000000
25%,125000.000000
50%,150000.000000
75%,150000.000000
max,150000.000000


Признаки `RegistrationMonth`, `NumberOfPictures`, `PostalCode` не информативны для дальнейшего обучения моделей, поэтому сразу их удалим. Также заодно удалим признаки с датами создания анкеты, последней активности пользователя и скачивания анкеты из базы.

In [21]:
df = df.drop(['RegistrationMonth', 'NumberOfPictures', 'PostalCode', 'DateCrawled', 'DateCreated', 'LastSeen'], axis=1)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 329635 entries, 0 to 354364
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             329635 non-null  int64 
 1   VehicleType       309738 non-null  object
 2   RegistrationYear  329635 non-null  int64 
 3   Gearbox           314090 non-null  object
 4   Power             329635 non-null  int64 
 5   Model             314053 non-null  object
 6   Kilometer         329635 non-null  int64 
 7   FuelType          305512 non-null  object
 8   Brand             329635 non-null  object
 9   Repaired          269827 non-null  object
dtypes: int64(4), object(6)
memory usage: 27.7+ MB


**Исследуем категориальные признаки**

In [23]:
# Посмотрим на типы кузова
df1 = df['VehicleType'].value_counts(dropna=False)
df1

sedan          89377
small          77999
wagon          63694
bus            28341
NaN            19897
convertible    19852
coupe          15627
suv            11778
other           3070
Name: VehicleType, dtype: int64

In [24]:
# Заполним пропущенные значения в типах кузова самым часто встречающимся значением
df['VehicleType'] = df['VehicleType'].fillna('sedan')

In [25]:
# Посмотрим на типы коробки передач
df1 = df['Gearbox'].value_counts(dropna= False)
df1

manual    251092
auto       62998
NaN        15545
Name: Gearbox, dtype: int64

In [26]:
# Заполним пропущенные значения в типах коробки передач самым часто встречающимся значением
df['Gearbox'] = df['Gearbox'].fillna('manual')

In [27]:
# Посмотрим на модели автомобилей
df1 = df['Model'].value_counts(dropna= False)
df1

golf                  26732
other                 23138
3er                   18670
NaN                   15582
polo                  12031
                      ...  
kalina                    6
rangerover                3
serie_3                   3
range_rover_evoque        2
serie_1                   1
Name: Model, Length: 251, dtype: int64

Для заполнения пропущенных значений в признаке `Model` у нас нет данных, поэтому удалим строки, в которых данное значение пропущено.

In [28]:
df = df[~(df['Model'].isna())]

Объединим в один признак бренд и модель автомобиля и запишем его в новую колонку. А колонки `Model` и `Brand` удалим.

In [29]:
df['NameAuto'] = df.loc[:,'Brand'] + '_' + df.loc[:,'Model']

In [30]:
df = df.drop(['Brand', 'Model'], axis=1)

In [31]:
# Исследуем типы топлива
df1 = df['FuelType'].value_counts(dropna= False)
df1

petrol      196824
gasoline     92404
NaN          19219
lpg           4717
cng            513
hybrid         198
other          114
electric        64
Name: FuelType, dtype: int64

In [33]:
# Заполним пропущенные значения в типах топлива самым часто встречающимся значением
df['FuelType'] = df['FuelType'].fillna('petrol')

In [34]:
# Исследуем факт ремонта автомобилей
df1 = df['Repaired'].value_counts(dropna= False)
df1

no     229893
NaN     53146
yes     31014
Name: Repaired, dtype: int64

In [35]:
# Заполним пропущенные значения заглушкой
df['Repaired'] = df['Repaired'].fillna('unknown')

In [36]:
# Проверим еще раз остались ли пропуски в значениях
df.isna().sum()

Price               0
VehicleType         0
RegistrationYear    0
Gearbox             0
Power               0
Kilometer           0
FuelType            0
Repaired            0
NameAuto            0
dtype: int64

**Вывод**

В изначальном датасете 354369 объектов. Целевой признак находится в столбце `Price`.

Была проведена предобработка данных:

1. Заполнены пропуски медианным значением и модой.
2. Обработаны выбросы и аномалии.
3. Удалены неинформативные признаки для дальнейшего обучения модели предсказания.

После всех манипуляций в таблице осталось 314053 объектов и 9 признаков. Удалению подверглось ~11.4% данных. Это приемлимо, так как данные изначально имели много пропусков.
В реальных условиях лучше было бы изначально восстановить данные.


## Обучение моделей

In [37]:
# Разделим данные на выборки. 60% данных отведем под тренировочную выборку df_train
df_train, df_vt = train_test_split(df, test_size=0.33, random_state=12345)

In [38]:
# оставшиеся данные поделим пополам на тестовую df_test и валидационную df_valid
df_test, df_valid = train_test_split(df_vt, test_size=0.5, random_state=12345)

In [39]:
# Проверим размеры получившихся таблиц 
df_train.shape, df_test.shape, df_valid.shape

((210415, 9), (51819, 9), (51819, 9))

In [40]:
# создадим переменные для признаков и целевого признака на тренировочной и валидационной выборках
features_train = df_train.drop(['Price'], axis=1)
target_train = df_train['Price']
features_valid = df_valid.drop(['Price'], axis=1)
target_valid = df_valid['Price']

In [41]:
# На тестовой выборке создадим переменные для признаков и целевого признака
features_test = df_test.drop(['Price'], axis=1)
target_test = df_test['Price']

In [42]:
# Создадим список с категориальными признаками
cat_features = ['VehicleType', 'Gearbox', 'FuelType', 'Repaired', 'NameAuto']

In [43]:
# Преобразуем тип признаков с object на category 
for col in cat_features:
    features_train[col] = features_train[col].astype('category')

In [44]:
for col in cat_features:
    features_valid[col] = features_valid[col].astype('category')

In [45]:
for col in cat_features:
    features_test[col] = features_test[col].astype('category')

**Обучим и подберем гиперпараметры для модели LightGBM**

In [73]:
model = LGBMRegressor()

param_lgbm = {
    'max_depth': [1,3,5,7,9,11],
    'n_estimators': [10,50,100,200]
}
random_search = RandomizedSearchCV(model, param_lgbm, n_iter=10, scoring='neg_mean_squared_error', cv=5, random_state=12345)
random_search.fit(features_train, target_train)
best_params = random_search.best_params_

best_model_lgbm = LGBMRegressor(**best_params)

Обучим модель с лучшими гиперпараметрами и измерим время обучения.

In [77]:
%%time
best_model_lgbm.fit(features_train, target_train)

CPU times: user 7.34 s, sys: 0 ns, total: 7.34 s
Wall time: 7.36 s


LGBMRegressor(max_depth=7, n_estimators=200)

Сделаем предсказание и измерим время.

In [78]:
%%time
pred = best_model_lgbm.predict(features_valid)

CPU times: user 903 ms, sys: 0 ns, total: 903 ms
Wall time: 871 ms


Измерим RMSE для LightGBM.

In [79]:
rmse = (mean_squared_error(target_valid, pred))**0.5

print('RMSE:', rmse)

RMSE: 1588.7198717397835


**Обучим и подберем гиперпараметры для модели Catboost**

In [46]:
model = CatBoostRegressor(loss_function='RMSE', verbose=False)

param_cat = {
    'learning_rate': [0.01, 0.05, 0.1],
    'depth': [1,3,5,7,9,11],
    'iterations': [10,50,100,200]
}
random_search = RandomizedSearchCV(model, param_cat, n_iter=10, scoring='neg_mean_squared_error', cv=5, random_state=12345)
random_search.fit(features_train, target_train, cat_features=cat_features)
best_params_cat = random_search.best_params_

best_model_cat = CatBoostRegressor(**best_params_cat)

Обучим модель с лучшими гиперпараметрами и измерим время обучения.

In [47]:
%%time
best_model_cat.fit(features_train, target_train, cat_features=cat_features, verbose=False)

CPU times: user 6.22 s, sys: 51.8 ms, total: 6.27 s
Wall time: 6.82 s


Сделаем предсказание и измерим время.

In [60]:
%%time
pred = best_model_cat.predict(features_valid)

CPU times: user 37 ms, sys: 3.83 ms, total: 40.9 ms
Wall time: 39 ms


Измерим RMSE для Catboost.

In [61]:
rmse_cat = (mean_squared_error(target_valid, pred))**0.5

print('RMSE:', rmse_cat)

RMSE: 1792.925167518895


**Обучим и подберем гиперпараметры для модели линейной регрессии**

In [53]:
# найдем категориальные признаки в таблице
ohe_features_ridge = features_train.select_dtypes(include='category').columns.to_list()
print(ohe_features_ridge)

['VehicleType', 'Gearbox', 'FuelType', 'Repaired', 'NameAuto']


In [54]:
# преобразуем категориальные признаки
encoder_ohe = OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False)
encoder_ohe.fit(features_train[ohe_features_ridge])

features_train[encoder_ohe.get_feature_names_out()] = encoder_ohe.transform(features_train[ohe_features_ridge])

# удаляем незакодированные категориальные признаки (изначальные колонки)
features_train = features_train.drop(ohe_features_ridge, axis=1)
features_train.head()

,RegistrationYear,Power,Kilometer,VehicleType_convertible,VehicleType_coupe,VehicleType_other,VehicleType_sedan,VehicleType_small,VehicleType_suv,VehicleType_wagon,...,NameAuto_volkswagen_up,NameAuto_volvo_850,NameAuto_volvo_c_reihe,NameAuto_volvo_other,NameAuto_volvo_s60,NameAuto_volvo_v40,NameAuto_volvo_v50,NameAuto_volvo_v60,NameAuto_volvo_v70,NameAuto_volvo_xc_reihe
340792,2005,145,150000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99722,2001,163,150000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
272367,1999,125,150000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
155447,2004,69,150000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45096,1997,105,150000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
features_valid[encoder_ohe.get_feature_names_out()] = encoder_ohe.transform(features_valid[ohe_features_ridge])
features_valid = features_valid.drop(ohe_features_ridge, axis=1)

/opt/conda/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [57]:
# Стандартизируем данные 
numeric = ['RegistrationYear', 'Power', 'Kilometer']

scaler = StandardScaler()
scaler.fit(features_train[numeric])

features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])

In [58]:
# Обучим и подберем гиперпараметры для модели линейной регрессии
model_lin = LinearRegression()

param_lin = {
    'fit_intercept': [True,False],
    'normalize': [True,False]
}

scorer = make_scorer(mean_squared_error, squared=False)

random_search = RandomizedSearchCV(model_lin, param_lin, n_iter=10, scoring=scorer, cv=5, random_state=12345)
random_search.fit(features_train, target_train)
best_params = random_search.best_params_

best_model_lin = LinearRegression(**best_params)

/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_search.py:306: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecate

Обучим модель с лучшими гиперпараметрами.

In [59]:
%%time
best_model_lin.fit(features_train, target_train)

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(


CPU times: user 17.1 s, sys: 6.48 s, total: 23.6 s
Wall time: 23.7 s


LinearRegression(normalize=True)

Сделаем предсказание.

In [60]:
%%time
pred = best_model_lin.predict(features_valid)

CPU times: user 83.7 ms, sys: 69.6 ms, total: 153 ms
Wall time: 173 ms


Найдем RMSE для линеййной регрессии.

In [61]:
rmse_lin = (mean_squared_error(target_valid, pred))**0.5

print('RMSE:', rmse_lin)

RMSE: 2611.154811534429


Были построены три модели для решения задачи регрессии (LightGBM, Catboost, линейная регрессия) для целевого признака `Price`. Их качество оценивалось метрикой RMSE. 

## Анализ моделей

In [48]:
# Соберем в сводную таблицу данные 
results = [
    ['LGBMRegressor', 7.36, 0.871, 1588.719], 
    ['CatBoostRegressor', 6.82, 0.039, 1792.925],
    ['LinearRegression', 23.7, 0.173, 2611.154]
]

columns = ['model','fit_time', 'pred_time', 'RMSE']

balanced_result = pd.DataFrame(data=results, columns=columns)
balanced_result

,model,fit_time,pred_time,RMSE
0,LGBMRegressor,7.36,0.871,1588.719
1,CatBoostRegressor,6.82,0.039,1792.925
2,LinearRegression,23.70,0.173,2611.154


## Тестирование лучшей модели

In [84]:
# Проверим качество модели LGBM на тестовой выборке
predictions_test = best_model_lgbm.predict(features_test)
print("RMSE на тестовой выборке:", (mean_squared_error(target_test, predictions_test))**0.5)

RMSE на тестовой выборке: 1592.269230342507


## Итоги исследования

Был проведен исследовательский анализ данных о технических характеристиках, комплектации и ценах автомобилей. Для построения модели определения цены на автомобиль.

Построены три модели для решения задачи регрессии (LightGBM, Catboost, линейная регрессия). Их качество оценивалось метрикой RMSE. 

Лучшее качество по метрике RMSE показали модели градиентного бустинга LGBM и CatBoost, со значениями 1588.719 и 1792.925 соответственно. При этом время на обучение и предскзание меньше затрачивает модель CatBoost, хотя разница с LGBM незначительна. Поэтому для введения в эксплуатацию я бы рекомендовала модель LGBM, так как она дает лучшее качество и не сильно отличается по времени работы от самой быстрой.

Также на тестовой выборке было проверено качество выбранной модели LGBM, RMSE = 1592.269.